# Chapter 2 : Modeling in the Frequency Domain (Part 1)
---

In [1]:
import sys

if (path := "C:/Users/Tom/pycharm-projects/python-control") not in sys.path:
    sys.path.append(path)

import sympy as sp

from python_control import (
    t, s,
    InverseLaplaceTransform, LaplaceTransform,
    DifferentialEquation,
    TransferFunction
)

> **Warning**<br>
> Always use the predefined symbols `t` and `s` from `python_control`. Do not redefine these symbols, as they must remain unique for the package to work properly.

## 2.2 : Laplace Transform Review

### Example 2.1 : Laplace Transform of a Time Function

$$f\left( t \right) = A{e^{ - at}} \cdot u\left( t \right)$$

Define the *Sympy* symbols used in the expression of function $f(t)$:

In [2]:
A, a = sp.symbols('A a')

Define the function $f(t)$:

In [3]:
f = A * sp.exp(-a * t) * sp.Heaviside(t)

This function $f(t)$ in the time domain is actually the *inverse Laplace transform* of the function $F(s)$ in the s-domain. To find $F(s)$, we can pass our function $f(t)$ to an instance of class `InverseLaplaceTransform`:

In [4]:
f = InverseLaplaceTransform(f)

Calling method `transform` on the `InverseLaplaceTransform` object, returns a `LaplaceTransform` object that holds the expression of $F(s)$ in the frequency domain:

In [5]:
F = f.transform()
F.expr

A/(a + s)

### Example 2.2 : Inverse Laplace Transform

Find the inverse Laplace transform of:
$$
F\left( s \right) = \frac{1}{{{{\left( {s + 3} \right)}^2}}}
$$

In [6]:
F = LaplaceTransform(1 / (s + 3) ** 2)
f = F.inverse()
f.expr

1.0*t*exp(-3.0*t)

### Example 2.3 : Laplace Transform Solution of a Differential Equation

Solve the differential equation for $y(t)$ if all initial conditions are zero.
$$
\frac{{{d^2}y}}{{d{t^2}}} + 12\frac{{dy}}{{dt}} + 32y = 32 \cdot u\left( t \right)
$$

The LHS of the differential equation is used to create a `DifferentialEquation` object. Parameter `f` accepts the name of the function $y(t)$ for which we need a solution. Parameters `coeffs` receives the coefficients of the differential equation inside a list. Through parameter `init_vals` the initial values are assigned (in a list sorted from highest to lowest order), but when  `None` (which is also the default argument) all intial values are considered equal to zero.

In [7]:
ode = DifferentialEquation(
    f='y',
    coeffs=[1, 12, 32],
    init_vals=None
)

Now we can solve the differential equation both in the time and frequency domain: 

In [8]:
Y, y = ode.solve(rhs=32 * sp.Heaviside(t))

`Y` refers to a `LaplaceTransform` object and `y` to an `InverseLaplaceTransform` object. To get the partial fraction expansion of `Y`, method `expanded` is called on this object, which returns a *Sympy* expression:

In [9]:
Y.expanded()

1/(s + 8) - 2/(s + 4) + 1/s

The *Sympy* expression of `y` can be accessed through its property `expr`: 

In [10]:
y.expr

1 - 2*exp(-4*t) + exp(-8*t)

## 2.3 : Transfer Function

### Example 2.4 : Transfer Function for a Differential Equation

Find the transfer function represented by:
$$
\frac{{dc\left( t \right)}}{{dt}} + 2c\left( t \right) = r\left( t \right)
$$
The transfer function $G(s)$ is the ratio of the output to the input of the system, i.e. $C(s) / R(s)$. This means that the right-hand side of the differential equation (i.e. the input-side of the system) will determine the numerator of the transfer function, and the left-hand side (i.e. the output-side of the system) will determine its denominator. 

It is fairly easy to write down the Laplace transformed representation of the differential equation: the coefficients in the differential equation will also be the coefficients in its Laplace transformed representation. 

In [11]:
G = TransferFunction.from_coefficients(num=[1], den=[1, 2])
G.expr

1.0/(1.0*s + 2.0)

### Example 2.5 : System Response from the Transfer Function

Use the result of example 2.4 to find the response to a unit step assuming zero initial conditions.

The unit step function (a.k.a *Heaviside* function) in the right-hand side of the differential equation is implemented in *Sympy* by the function `Heaviside`.

In [12]:
r = sp.Heaviside(t)

Now, we need the Laplace transform of this function. For this we can put the function in a `InverseLaplaceTransform` object and then call its `transform` method, which will return a `LaplaceTransform` object: 

In [13]:
R = InverseLaplaceTransform(r).transform()

The transfer function object `G` was already created in the previous example. To get the response of the system, we need to call the method `response` on this object, passing it our  `LaplaceTransform` object:

In [14]:
C = G.response(R)
C.expanded()

-1/(2*(s + 2)) + 1/(2*s)

In [15]:
c = C.inverse()
c.expr

0.5 - 0.5*exp(-2.0*t)

### Skill-Assessment Exercise 2.3

Find the transfer function corresponding to the differential equation:
$$
\frac{{{d^3}c}}{{d{t^3}}} + 3\frac{{{d^2}c}}{{dt}} + 7\frac{{dc}}{{dt}} + 5c = \frac{{{d^2}r}}{{dt}} + 4\frac{{dr}}{{dt}} + 3r
$$

In [16]:
G = TransferFunction.from_coefficients(num=[1, 4, 3], den=[1, 3, 7, 5])
G.expr

(1.0*s + 3.0)/(1.0*s**2 + 2.0*s + 5.0)

### Skill-Assessment Exercise 2.5

Find the ramp response for a system whose transfer function is:
$$
G\left( s \right) = \frac{s}{{\left( {s + 4} \right)\left( {s + 8} \right)}}
$$

The ramp function can be programmed as:

In [17]:
r = t * sp.Heaviside(t)

Its Laplace transform is:

In [18]:
R = InverseLaplaceTransform(r).transform()
R.expr

s**(-2)

The transfer function $G(s)$ can be implemented with:

In [19]:
G = TransferFunction(s / ((s + 4) * (s + 8)))

The ramp response of the system in the frequency domain can now be retrieved with:

In [20]:
C = G.response(R)
C.expanded()

1/(32*(s + 8)) - 1/(16*(s + 4)) + 1/(32*s)

Calling the `inverse` method on the `LaplaceTransform` object `C`, gives us back an `InverseLaplaceTransform` object:

In [21]:
c = C.inverse()
c.expr

0.03125 + 0.03125*exp(-8.0*t) - 0.0625*exp(-4.0*t)